In [ ]:

import random
import pandas as pd

RANDOM_SEED = 97

INPUT_DATASET = "input_dataset.csv"
OUTPUT_DATASET = "output_dataset.csv"

BALANCE_COL = "working"
VALUES = [True, False]

# set the random seed for reproducibility
random.seed(97)

# load the dataset
dataset = pd.read_csv(INPUT_DATASET)

# figure out the minimum number of the values
value_counts = []
for value in VALUES:
    value_counts.append(dataset[dataset[BALANCE_COL] == value].shape[0])
min_num_rows = min(value_counts)
for index, value in enumerate(VALUES):
    print(f"There were {value_counts[index]} {value}s in the dataset - the kept amount is {min_num_rows}.")

# randomly select the minumum number of rows each of the values
chosen_ids = []
for label in VALUES:
    ids = dataset[dataset[BALANCE_COL] == label].index
    chosen_ids.extend(random.sample(list(ids), min_num_rows))

# remove the non-chosen ids from the dataset
dataset = dataset.drop(dataset.index[list(set(range(dataset.shape[0])) - set(chosen_ids))])

# save the dataset
dataset.to_csv(OUTPUT_DATASET, index=False)

In [2]:
import pandas as pd
from pyarrow.parquet import read_table
from scipy.sparse import coo_matrix

In [3]:
def load_data(data_path, row_info_path, column_info_path):
    # Load non_zero parquet data
    table = read_table(data_path)
    nonzero_data = table.to_pandas()
    
    # Adjust column indices to be 0-based
    nonzero_data['col_indices'] = nonzero_data['col_indices'] - 1
    
    # Load row and column index info
    rows = pd.read_csv(row_info_path)
    row_names = rows.iloc[:, 1].to_list()
    
    columns = pd.read_csv(column_info_path)
    column_names = columns.iloc[:, 1].to_list()
    
    # Convert the sparse matrix to a dense DataFrame
    sparse_matrix = coo_matrix(
        (nonzero_data['nonzero_elements'], (nonzero_data['row_indices'], nonzero_data['col_indices'])),
        shape=(len(row_names), len(column_names))
    )
    
    
   

    print('Returning sparse_matrix, column_names, and row_names')
    
    return sparse_matrix, column_names, row_names

In [5]:
metadata = pd.read_csv('C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\GSE185948_metadata_RNA.csv')

In [7]:
metadata.head()

,name,barcode,patient,gender,disease,celltype,nCount_RNA,nFeature_RNA,UMAP_1,UMAP_2
0,PKD_ACACGCGGTATCGGTT-1_1,ACACGCGGTATCGGTT-1,PKD1,female,PKD,TAL1,1234.684629,1250,-7.416020,-6.008874
1,PKD_ACACGCGGTTTGGCTA-1_1,ACACGCGGTTTGGCTA-1,PKD1,female,PKD,PT2,1865.542588,1650,2.499409,-6.587287
2,PKD_ACACGCGTCATGTCTT-1_1,ACACGCGTCATGTCTT-1,PKD1,female,PKD,CNT_PC,1812.700523,1419,-2.254505,8.526364
3,PKD_ACACTGAAGACCCTTA-1_1,ACACTGAAGACCCTTA-1,PKD1,female,PKD,FIB,978.772591,1089,9.949437,2.086737
4,PKD_ACACTGAAGCGACAGT-1_1,ACACTGAAGCGACAGT-1,PKD1,female,PKD,PT1,2361.558871,2013,7.886921,-8.587954


In [18]:
demo = metadata[['gender', 'disease', 'name']]
min = demo.groupby(['gender', 'disease']).count().reset_index()['name'].min()

In [23]:
data = []
for name, group in demo.groupby(['gender', 'disease']):
    df = group
    data.append(df['name'].sample(n = 3))

In [25]:
data[0]

18639    PKD_TCTGCCATCGAACCTA-1_2
990      PKD_AGGTTACTCCTGCTAC-1_1
39526    PKD_AGCTTCCAGTACAGAT-1_5
Name: name, dtype: object

In [4]:
data_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\non_zero.parquet'
row_info_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\row_names.csv'
column_info_path = 'C:\\Users\\nphda\\OneDrive\\Desktop\\Data Mining\\Jackson-and-Anh-Data-Mining-Assingment-4\\data\\col_names.csv'
sparse_matrix, row_names, column_names = load_data(data_path, row_info_path, column_info_path)

Returning sparse_matrix, column_names, and row_names


In [6]:
csr_matrix = sparse_matrix.tocsr()